<a href="https://colab.research.google.com/github/sganguly3000ai/makemore/blob/main/makemore_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bigram character level language model using neural network**

In [2]:
import torch
import numpy as np
import string
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [3]:
chars = list(string.ascii_lowercase)
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [127]:
words = ['sumit','sorit','swapna','suraj','sujit', 'sujoy', 'manoj', 'amit',
         'kajol', 'steve', 'andrew','bob', 'cindy', 'ebrahim', 'daku', 'johny',
         'timmy', 'david', 'tarun', 'nancy', 'edith', 'jason', 'kyle', 'joseph',
         'kimberly', 'emma', 'irene', 'michael', 'scott', 'valentino']
xs, ys = [], []
N = torch.zeros((27,27) , dtype = torch.int32)
for w in words[:5]:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    print (ch1, ch2)
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs) # inputs
ys = torch.tensor(ys) # labels
num = xs.nelement()   # keeping track of how many bigrams
print('number of examples or inputs: ', num)

"""
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

                INITIALIZE THE NEURAL NETWORK WITH 27 NEURON IN 1 LAYER

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

"""

# RANDOMLY INITIALIZE 27 NEURONS' WEIGHTS. EACH NEURON RECEIVES 27 INPUTS
g = torch.Generator().manual_seed(2147483647)

"""
IMPORTANT TO SET THE requires_grad=True : SO THAT PYTORCH CAN KEEP TRACK OF THE GRADIENTS OF W
"""

# getting random numbers from a normal distribution
W = torch.randn((27,27), generator=g, requires_grad=True)


. s
s u
u m
m i
i t
t .
. s
s o
o r
r i
i t
t .
. s
s w
w a
a p
p n
n a
a .
. s
s u
u r
r a
a j
j .
. s
s u
u j
j i
i t
t .
number of examples or inputs:  31


In [112]:
W.shape
xenc = F.one_hot(xs,num_classes=27).float()
logits = xenc @ W
logits.shape

torch.Size([188, 64])

In [131]:
for k in range(100000):
  """
  +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

                                      FORWARD PASS
                                    GRADIENT DESCENT

  +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

  """

  # cannot pass just one integer as an input to a neural network but can pass
  # vectors
  # thats why using one hot encoding to generate a tensor represented by 1
  # at the index represented by the xs integer

  # should not pass in vectors of dtype as int, need to convert xenc to float

  xenc = F.one_hot(xs,num_classes=27).float() #inputs to the network: one hot encoding
  # plt.imshow(xenc)  uncomment to visualize in a plot

  # multiplying to get the logits from the neurons
  # feeding in all 6 inputs simultaneously into 27 neurons in layer 1
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # equivalent to the N tensor we used in makemore where we were just counting the bigrams
  probs = counts / counts.sum(1, keepdims = True) # probabilities for next character (of 27 characters)

  # btw: the last 2 lines here are together called a 'softmax'

  """
  +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

      CALCULATING THE LOSS - USING NEGATIVE LOG LIKELIHOOD TO CALCULATE LOSS SINCE THIS IS CLASSIFICATION

                IF THIS WAS REGRESSION WE WOULD USE MEAN SQUARED ERROR TO CALCULATE LOSS

  +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

  plucking the values of the probabilities for 6 indices corresponding to the labels
  from the probs tensor (since there are 6 labels)
  """

  loss = -probs[torch.arange(num), ys].log().mean()
  """
  +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

                                    BACKWARD PASS


                                WHAT DOES backward() DO?

    pytorch does a backward pass through the neural network starting at the loss node
    pytorch keeps track of the gradients which are calculated by performing the derivatives
    at each node

  +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

  """
# SETTING GRADIENTS OF THE WEIGHTS TO ZERO OR NONE

  W.grad = None
  loss.backward()

  """
  +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

                                    UPDATE W


                                WHAT DOES backward() DO?

    pytorch does a backward pass through the neural network starting at the loss node
    pytorch keeps track of the gradients which are calculated by performing the derivatives
    at each node
    we update the weights (in the negative direction of the gradients) with a learning factor
    multiplied by the gradient

  +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

  """
  W.data += -.01 * W.grad

In [129]:
print(loss.item())

0.47739189863204956


In [106]:
counts.shape

torch.Size([12, 64])

In [107]:
probs[0]

tensor([5.0387e-05, 1.2137e-05, 1.4665e-05, 5.4416e-06, 1.9284e-05, 1.4635e-05,
        3.5300e-06, 2.5153e-05, 1.6116e-05, 3.1871e-05, 9.8211e-06, 2.9090e-05,
        1.1164e-05, 1.0174e-05, 4.4541e-05, 9.3975e-05, 5.0209e-05, 3.2678e-06,
        2.6647e-05, 9.9870e-01, 3.3867e-05, 1.3154e-05, 3.7704e-06, 2.2124e-05,
        1.4107e-05, 8.0990e-05, 7.9331e-05, 8.1644e-06, 4.8511e-06, 1.9571e-05,
        5.5916e-06, 2.7760e-05, 1.5722e-05, 4.2685e-05, 7.2105e-06, 6.8890e-06,
        2.7372e-06, 4.1047e-05, 4.8246e-06, 4.0039e-05, 6.1989e-06, 1.2194e-05,
        1.0969e-05, 2.0065e-05, 4.3762e-06, 3.7460e-05, 2.4982e-05, 2.2312e-05,
        1.5762e-05, 2.9223e-06, 1.6640e-05, 2.9466e-05, 2.3884e-05, 5.1169e-06,
        1.7111e-05, 5.9906e-06, 2.0889e-05, 2.2588e-05, 6.5089e-06, 7.3421e-06,
        1.6925e-05, 1.2424e-05, 5.7653e-06, 3.5657e-06],
       grad_fn=<SelectBackward0>)

In [55]:
torch.multinomial(probs[4], num_samples=27, replacement=True, generator=g)

tensor([20, 20, 10, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        20, 20, 20, 20, 20, 20, 20, 20, 20])

In [ ]:
"""
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

                     VISUALIZING WHERE THE LOSS IS COMING FROM

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

"""

nlls = torch.zeros(6) # negetive log likelihood

for i in range(6):
  # i-th bigram:
  x = xs[i].item() # input character index
  y = ys[i].item() # label character index
  print('================================================================\n')
  print(f'bigram example {i+1}: {itos[x]}{itos[y]} (indexes {x},{y})')
  print('input to the neural net:', x)
  print('output probabilities from the neural net:', probs[i])
  print('label (actual next character):', y)
  p = probs[i, y]
  print('probability assigned by the network to the correct character:', p.item())
  logp = torch.log(p)
  print('log likelihood:', logp.item())
  nll = -logp
  print(f'negative log likelihood: {nll.item()} \n')
  nlls[i] = nll
  print('================================================================\n')
  print(f'average negative log likelihood after each bigram, i.e. loss = {nlls.mean().item()} \n')
  print('================================================================\n')

print(f'average negative log likelihood after all inputs, i.e. loss = {nlls.mean().item()} \n')

In [132]:
xs

tensor([ 0, 19, 21, 13,  9, 20,  0, 19, 15, 18,  9, 20,  0, 19, 23,  1, 16, 14,
         1,  0, 19, 21, 18,  1, 10,  0, 19, 21, 10,  9, 20])

# Sampling


*   torch.multinomial generates sample integer based on probabilities that are fed to the function

*   These integers are the index to the probability tensor P


*   The out list is being appended with the character equivalent of the index number thst the model is generating based on its training. The generation stops when the index value generated is 0 (indicating the end of a generated name).









In [130]:
g = torch.Generator().manual_seed(2147483647)
print (f'GENERATED NAMES BY THE BIGRAM MODEL')
print (f'===================================\n')
for i in range(num):
  p = probs[i]
  ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
  print ('Predicted character= ' , itos[ix])

GENERATED NAMES BY THE BIGRAM MODEL

Predicted character=  s
Predicted character=  u
Predicted character=  m
Predicted character=  i
Predicted character=  t
Predicted character=  .
Predicted character=  s
Predicted character=  u
Predicted character=  r
Predicted character=  i
Predicted character=  t
Predicted character=  .
Predicted character=  s
Predicted character=  o
Predicted character=  a
Predicted character=  p
Predicted character=  n
Predicted character=  a
Predicted character=  p
Predicted character=  s
Predicted character=  o
Predicted character=  j
Predicted character=  a
Predicted character=  .
Predicted character=  i
Predicted character=  s
Predicted character=  w
Predicted character=  m
Predicted character=  i
Predicted character=  t
Predicted character=  .


In [43]:
ix

19

#Find the loss

In [12]:
log_likelihood = 0.0
n = 0
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1,ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
    print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print (f'{log_likelihood=}')
#negetive log_likelihood
nll = -log_likelihood
print (f'{nll=}')
print (f'{nll/n}') # loss

.s: 0.2667 -1.3218
su: 0.4000 -0.9163
um: 0.1667 -1.7918
mi: 0.3000 -1.2040
it: 0.3846 -0.9555
t.: 0.4545 -0.7885
.s: 0.2667 -1.3218
so: 0.2000 -1.6094
or: 0.1000 -2.3026
ri: 0.1429 -1.9459
it: 0.3846 -0.9555
t.: 0.4545 -0.7885
.s: 0.2667 -1.3218
sw: 0.1000 -2.3026
wa: 0.5000 -0.6931
ap: 0.0625 -2.7726
pn: 0.5000 -0.6931
na: 0.1667 -1.7918
a.: 0.1250 -2.0794
.s: 0.2667 -1.3218
su: 0.4000 -0.9163
ur: 0.1667 -1.7918
ra: 0.2857 -1.2528
aj: 0.1250 -2.0794
j.: 0.2500 -1.3863
.s: 0.2667 -1.3218
su: 0.4000 -0.9163
uj: 0.3333 -1.0986
ji: 0.1250 -2.0794
it: 0.3846 -0.9555
t.: 0.4545 -0.7885
.s: 0.2667 -1.3218
su: 0.4000 -0.9163
uj: 0.3333 -1.0986
jo: 0.5000 -0.6931
oy: 0.1000 -2.3026
y.: 0.8571 -0.1542
.m: 0.0667 -2.7081
ma: 0.2000 -1.6094
an: 0.1875 -1.6740
no: 0.1667 -1.7918
oj: 0.1000 -2.3026
j.: 0.2500 -1.3863
.a: 0.0667 -2.7081
am: 0.0625 -2.7726
mi: 0.3000 -1.2040
it: 0.3846 -0.9555
t.: 0.4545 -0.7885
.k: 0.1000 -2.3026
ka: 0.2500 -1.3863
aj: 0.1250 -2.0794
jo: 0.5000 -0.6931
ol: 0.1000 -

# Check the probability and loss for any given word

In [25]:
log_likelihood = 0.0
n = 0
for w in ['steve']:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1,ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
    print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print (f'{log_likelihood=}')
#negetive log_likelihood
nll = -log_likelihood
print (f'{nll=}')
print (f'{nll/n}') # loss

.s: 0.2667 -1.3218
st: 0.1000 -2.3026
te: 0.0909 -2.3979
ev: 0.0769 -2.5649
ve: 0.3333 -1.0986
e.: 0.2308 -1.4663
log_likelihood=tensor(-11.1521)
nll=tensor(11.1521)
1.858689308166504


# Model smoothing


*   The loss is infinite for some names that the model has 0 probability of predicting
*   In order to alleviate this infinite loss this model can be smoothed out by adding some positive values to the probability



In [29]:

# here we are adding 1 to every bigram count
# any number can be added
P = (N+1).float()
D = P.sum(1, keepdim = True)
# If the sum of the items in the row is 0, then set it to 1 to avoid division by 0
for d in D:
  if d[0]==0.0:
    d[0]= 1.0
P /= D

g = torch.Generator().manual_seed(2147483647)
print (f'GENERATED NAMES BY THE BIGRAM MODEL')
print (f'===================================\n')
for i in range(10):
  out = []
  ix = 0
  while True:
    p = P[ix]
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print (''.join(out))

log_likelihood = 0.0
n = 0
for w in ['chico']:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1,ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
    print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print (f'{log_likelihood=}')
#negetive log_likelihood
nll = -log_likelihood
print (f'{nll=}')
print (f'{nll/n}') # loss

GENERATED NAMES BY THE BIGRAM MODEL

juwjdvditkaqaz.
p.
cfqywacny.
kuitrltohcogsjgwzvudahntauy.
bilevhajkdbdainrwimbl.
snjyinaylaftezffvmumthyfodtumj.
pfytsuwjhruanq.
core.
ysezocfky.
jabdywebfmiifmwyfin.
.c: 0.0351 -3.3499
ch: 0.0645 -2.7408
hi: 0.0625 -2.7726
ic: 0.0500 -2.9957
co: 0.0645 -2.7408
o.: 0.0541 -2.9178
log_likelihood=tensor(-17.5177)
nll=tensor(17.5177)
2.9196126461029053


# Complete model code

In [ ]:
# Import all required packages

import torch
import numpy as np
import string
import matplotlib.pyplot as plt

# Create dictionaries as character lookup and translation to integer
# This is required since strings cannot be stored in tensors

chars = list(string.ascii_lowercase)
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

# Create a list of common names. Preferably one should find a dataset of names online

words = ['sumit','sorit','swapna','suraj','sujit', 'sujoy', 'manoj', 'amit',
         'kajol', 'steve', 'andrew','bob', 'cindy', 'ebrahim', 'daku', 'johny',
         'timmy', 'david', 'tarun', 'nancy', 'edith', 'jason', 'kyle', 'joseph',
         'kimberly', 'emma', 'irene', 'michael', 'scott', 'valentino']

# Tensor N will hold the raw counts of the Bigram

N = torch.zeros((27,27) , dtype = torch.int32)
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1,ix2] += 1

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Reds')

for i in range(27):
  for j in range(27):
    chstr = itos[i] + itos [j]
    plt.text(j, i, chstr, ha="center", va="bottom", color="gray")
    plt.text(j, i, N[i,j].item(), ha="center", va="top", color="gray")
plt.axis('off')

P = N.float()
D = P.sum(1, keepdim = True)
# If the sum of the items in the row is 0, then set it to 1 to avoid division by 0
for d in D:
  if d[0]==0.0:
    d[0]= 1.0
P /= D

g = torch.Generator().manual_seed(2147483647)
print (f'GENERATED NAMES BY THE BIGRAM MODEL')
print (f'===================================\n')
for i in range(10):
  out = []
  ix = 0
  while True:
    p = P[ix]
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print (''.join(out))

log_likelihood = 0.0
n = 0
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1,ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
    print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print (f'{log_likelihood=}')
#negetive log_likelihood
nll = -log_likelihood
print (f'{nll=}')
print (f'{nll/n}') # loss